# How to build a training set (Example - LiNiO$_2$)
Current functionality only allows input from VASP vasprun.xml files. This guide is for creating a training set in VASP only. You will require a [VASP license](https://www.vasp.at) and a HPC to run the calculations.

## Material considerations
The choice of settings you use is vasp are highly dependent on the material. Before starting consider the material properties, i.e. magnetic properties, and whether including of dispersion is needed. These will need to be set in the `INCAR` file during your calculations. Details of exact `INCAR` flags are not discussed here and more details can be found on the [vasp wiki](https://www.vasp.at/wiki/index.php/The_VASP_Manual).

Before starting, also consider if you want to include defects in your training set. For example, having a training set consisting of pristine structures and those with single vacancies present may result is a slightly higher error, but overall produce a better representative interatomic potential for looking at defect chemistry. In this case, you would do the following steps on the additional structures as well as the pristine.

## VASP input files

**Structural file** - The `POSCAR` file contains the structural information. This contains the cell dimensions, list of species labels and number of each species, and a list of the x,y,z coordinates in either fractional or cartesian coordinates.

**Pseudopotentials** - The `POTCAR` file contains the pseudopotential for each atomic species used in the calculation. For more than one species the individual `POTCAR` files need to be concatenated into a single `POTCAR` files of the species, in the order the species appear in the `POSCAR` file i.e. if they are listed in the `POSCAR` as Li, Ni, O then the same order is needed in the `POTCAR`.

**Sampling file** - The `KPOINTS` file contains the details for the Bloch vectors (k-points) that will be used to sample the Brillouin zone in your calculation. These can be specified in different ways. More details can be found on the wiki [KPOINTS](https://www.vasp.at/wiki/index.php/KPOINTS) page. This file is not always essential to the calculation as the [KSPACING](https://www.vasp.at/wiki/index.php/KSPACING) flag in the `INCAR` can be used instead. In this guide, the `KSPACING` flag is used in the `INCAR` in place of the `KPOINTS` file.

**Calculation details** - The `INCAR` file is the central input file for VASP and determines "what to do and how to do it". There is a large number of parameters which can be set here. This is where you set your calculation mode, such as the electronic relaxation conditions, ionic relaxation conditions, and of course, the settings needed for your specific material. Changes to the `INCAR` file for each stage of building the training set are discussed here.

**Submission file** - This will be unique to your HPC and you should refer to your HPC userguide.


## Stages of building the training set

There are 4 stages in building the training set. These are:
1. Convergence testing
2. Structural relaxation
3. Thermal distortion
4. High precision single step calculations

### Convergence testing

This stage is used to check that your settings are properly converged. Common tests are to converge with respect to `ENCUT` and `KSPACING`. Below is an example `INCAR` for LiNiO$_2$. During convergence testing both `ENCUT` and `KSPACING` were varied independently. `ENCUT` was varied between 400-540 in steps of 20. `KSPACING` was varied between 0.2-0.4 steps of 0.05. These are then plotted, compared, and the favourable value chosen.

```
Start parameter for this run:
ISTART  =           1         ! start job from scratch (see INIWAV)

Electronic relaxation:
ENCUT    =        400
ALGO     =        ALL

Calculation mode:
PREC     =   Accurate        ! Calculation level (Changes FFT-grids)
LREAL    =    .FALSE.
GGA      =         PS        !PS = PBESol
EDIFF    =       1E-4

Ionic relaxation:
NSW      =          0

k-sampling:
KSPACING =       0.30
```

### Structural relaxation

This is the first stage in the calculations. This is performed to obtain the ground state relaxed geometry of the system. For LiNiO$_2$ the magnetic spins need to be considered and the choice of functional is GGA+U. The example `INCAR` shown below is with these additional commands removed to highlight the base parameters which are needed. Note: the choice of `ISIF` setting determines what parts of the system are relaxed, such as atoms only, or cell relaxation as well, etc. Further details on this are given in the [vasp wiki ISIF](https://www.vasp.at/wiki/index.php/ISIF) page.

```
Start parameter for this run:
ISTART  =           1         ! start job from scratch (see INIWAV)

Electronic relaxation:
ENCUT    =        700
ALGO     =        ALL

Calculation mode:
PREC     =   Accurate        ! Calculation level (Changes FFT-grids)
LREAL    =    .FALSE.
GGA      =         PS        !PS = PBESol
EDIFF    =       1E-4

Ionic relaxation:
NSW      =        100
IBRION   =          2
ISIF     =          3

k-sampling:
KSPACING =       0.30
```

### Thermal distortion

The second stage in the calculation is performing high temperature molecular dynamics to thermally distort the structure. This is done in three steps:
1. Equilibration using NVE
2. Equilibration using NVT
3. Production run using NVT

The equilibration steps are used to adjust the systems to the production conditions. The thermostat is added during the NVT equilibration and run for a number of steps till the system and temperature have settled. In the example `INCAR` below, changing between these ensembles is simple; For NVE `SMASS = -1` is used, whereas for NVT `SMASS = 3` is used. These setting are shown in the final two blocks headed with `---- <header> ----`. Just include the relevant block for each step. In addition to this, the final production run will need to run for a longer time period. This is done by changing `NSW`. For this example, in the production run `NSW = 10000`.

The temperature of the system should be set high enough to thermally distort the material within the timeframe, but not to melt the structure all together. For LiNiO$_2$ this was set to 1000 using `TEBEG` and `TEEND`, which set the start and final temperatures.

```
Start parameter for this run:
ISTART  =           1         ! start job from scratch (see INIWAV)
ISIF    =           2

Electronic relaxation:
ENCUT    =        700
ALGO     =  Very Fast

Calculation mode:
PREC     =     Normal        ! Calculation level (Changes FFT-grids)
LREAL    =       Auto
GGA      =         PS        !PS = PBESol
EDIFF    =       1E-6

k-sampling:
KSPACING =       0.30

MD (do little writing to save disc space):
IBRION   =          0
NSW      =       1000
NWRITE   =          0
LCHARG   =    .FALSE.
LWAVE    =    .FALSE.
TEBEG    =       1000
TEEND    =       1000

---- NVE with periodic T rescaling every 50 steps ----
SMASS    =         -1
NBLOCK   =         50
POTIM    =        1.5

---- canonic (Nose) MD with XDATCAR updated every 50 steps (NVT) ----
SMASS    =          3
NBLOCK   =         50
POTIM    =        1.5
```

### High precision single step calculations

To create the training set for the potential fit, you need to take frames from the thermally distorted MD trajectory in the previous step. This is to sample structures which occur during molecular dynamics. The trajectory is stored in the `XDATCAR` file. There are multiple tools available to extract a single structure file from the `XDATCAR`, one of which is through using pymatgen, an example of which is:

```
#! /usr/bin/env python3
from pymatgen.io.vasp.outputs import Xdatcar

def xdatcar_to_poscar():
   in_file = Xdatcar('XDATCAR')
   psocar = in_file.structures[0]
   poscar.to(filename='POSCAR', fmt='POSCAR')

if __name__ == '__main__':
   xdatcar_to_poscar()
```

This script extracts a single trajectory frame, currently set to the first frame using `[0]`. Depending on how large you want your training set, you will need to sample the trajectory appropriately (and/or run the thermal distortion for a longer time period). Once you have collected a number of frames as `POSCAR` structure files, you then need to conduct a final, high precision, single step calculation on each of them. This is to calculate the forces and stress tensors. An example `INCAR` for this is given below. It is similar to the structural relaxation step, with the addition of a single ionic relaxation step i.e. the atoms are not moving. Once you have done this, the output `vasprun.xml` files for each will be your training set. 

```
Start parameter for this run:
ISTART  =           1         ! start job from scratch (see INIWAV)

Electronic relaxation:
ENCUT    =        700
ALGO     =        ALL

Calculation mode:
PREC     =   Accurate        ! Calculation level (Changes FFT-grids)
LREAL    =    .FALSE.
GGA      =         PS        !PS = PBESol
EDIFF    =       1E-6
EDIFFG   =      -0.01

Ionic relaxation:
NSW      =          0
IBRION   =          2
ISIF     =          2

k-sampling:
KSPACING =       0.30
```


## Formatting and save location

The `vasprun.xml` files are placed in the `vasprun` directory and are labelled as `vasprunX.xml` where X is an integer increasing from 0. For example, a training set of 4 structures would have 4 vasprun.xml files labelled as vasprun0.xml, vasprun1.xml, vasprun2.xml, and vasprun3.xml. Additional files and saved training sets can be stored in a subdirectory of the `vasprun` directory as the potential fitting code will only go to a depth of 1.